### Compiled models for sentiment analysis
- First portion for headlines, second for abstract

In [1]:
#import dependencies
import pandas as pd
import numpy as np 
import json

#### Load LLM JSON data

In [2]:
# load json 
path = '../../../data/clean/multimodal_inputs/llm_text_data_processed_abstract_AAPL.json'
with open(path,"r") as file:
    data = json.load(file)

df = pd.DataFrame(list(data.items()), columns=['pub_date','abstract'])
df = df.explode('abstract')
df.head()

,pub_date,abstract
0,2015-01-05,You can adjust your settings so that only cert...
0,2015-01-05,It’s time to take note of what lies at the top...
1,2015-01-06,NaN
2,2015-01-07,"With apps able to act as cameras, media player..."
3,2015-01-08,Google’s app store is the largest in the world...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3402 entries, 0 to 2513
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pub_date  3402 non-null   object
 1   abstract  2143 non-null   object
dtypes: object(2)
memory usage: 79.7+ KB


In [4]:
# Add columns
df['ticker'] = 'AAPL'

# Replace NaN with empty string for models to process
df['abstract'] = df['abstract'].fillna("")
df.reset_index(drop=True, inplace=True)
df

,pub_date,abstract,ticker
0,2015-01-05,You can adjust your settings so that only cert...,AAPL
1,2015-01-05,It’s time to take note of what lies at the top...,AAPL
2,2015-01-06,,AAPL
3,2015-01-07,"With apps able to act as cameras, media player...",AAPL
4,2015-01-08,Google’s app store is the largest in the world...,AAPL
...,...,...,...
3397,2024-12-23,,AAPL
3398,2024-12-24,,AAPL
3399,2024-12-26,,AAPL
3400,2024-12-27,,AAPL


#### [Unused] Load company data

In [3]:
#edit this part for new company
df = pd.read_csv('Apple_Inc_text_data.csv')
# format datetime again
df['pub_date'] = pd.to_datetime(df['pub_date']) 
df['pub_date'] = df['pub_date'].dt.date
#adding ticker in case we want to merge any DFs
df['ticker'] = 'AAPL'
df = df[['ticker'] + [col for col in df.columns if col != 'ticker']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ticker            2143 non-null   object
 1   pub_date          2143 non-null   object
 2   abstract          2143 non-null   object
 3   lead_para         2124 non-null   object
 4   headline          2143 non-null   object
 5   doc_type          2143 non-null   object
 6   section_name      2143 non-null   object
 7   type_of_material  2142 non-null   object
 8   rank              2143 non-null   int64 
 9   web_url           2143 non-null   object
dtypes: int64(1), object(9)
memory usage: 167.6+ KB


In [4]:
#narrowing down to headlines and abstract
# Select the desired columns
sentiment_df = df[['ticker', 'pub_date', 'headline']]
sentiment_df = sentiment_df.copy()  # Make a copy to avoid modifications on a slice


## [Unused] Run headlines through all 5 models

- This section compiles all sentiment features for all 5 models for 1 company
- The final dataframe should consist of positive, neagtive and neutral probabilities for all models

### 1. Vader

In [5]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Tammy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
# Run model
sia = SentimentIntensityAnalyzer()

# Function to get full sentiment scores (including probabilities)
def get_sentiment(text):
    # Get the sentiment scores dictionary (positive, neutral, negative, and compound scores)
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores

# Apply the sentiment analysis to the 'headline' column
sentiment_df['sentiment_scores'] = sentiment_df['headline'].apply(get_sentiment)

# Separate out the individual sentiment probabilities into new columns
sentiment_df['vader_pos'] = sentiment_df['sentiment_scores'].apply(lambda x: x['pos'])
sentiment_df['vader_neu'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neu'])
sentiment_df['vader_neg'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neg'])

# Function to classify sentiment into Positive, Negative, or Neutral based on the compound score
def classify_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the compound score to classify sentiment
sentiment_df['vader_label'] = sentiment_df['sentiment_scores'].apply(lambda x: classify_sentiment(x['compound']))

# Display the result
# Drop the 'sentiment_scores' column from the DataFrame
sentiment_df = sentiment_df.drop(columns=['sentiment_scores'])

# Display the result
sentiment_df.head(5)


,ticker,pub_date,headline,vader_pos,vader_neu,vader_neg,vader_label
0,AAPL,2015-04-07,M.B.A. Programs That Get You Where You Want to Go,0.126,0.874,0.0,Positive
1,AAPL,2015-04-14,What We’re Reading,0.000,1.000,0.0,Neutral
2,AAPL,2015-04-13,IBM Creates Watson Health to Analyze Medical Data,0.231,0.769,0.0,Positive
3,AAPL,2015-04-22,What’s That on Beyoncé’s Wrist? Let Me Guess ....,0.000,1.000,0.0,Neutral
4,AAPL,2015-04-01,Daily Report: Tech Leaders Come Together to Op...,0.000,1.000,0.0,Neutral


### 2. Transformer (distilRoBERTa)

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("frostedtrees/Fin_distilroberta")
drob_model = AutoModelForSequenceClassification.from_pretrained("frostedtrees/Fin_distilroberta", num_labels=3)  # Assuming 3 sentiments: positive, neutral, negative


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def predict_sentiment(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = drob_model(**inputs)
    
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)  # Convert logits to probabilities
    
    return probs[0].cpu().numpy()  # Returns an array of probabilities

# Apply prediction to DataFrame
sentiment_probs = sentiment_df['headline'].apply(predict_sentiment)

# Create separate columns for sentiment probabilities
sentiment_df[['drob_neg', 'drob_neu', 'drob_pos']] = pd.DataFrame(sentiment_probs.tolist(), index=sentiment_probs.index)

# Determine final sentiment label based on highest probability and simplify label mapping
sentiment_df['drob_label'] = sentiment_df[['drob_neg', 'drob_neu', 'drob_pos']].idxmax(axis=1)
sentiment_df['drob_label'] = sentiment_df['drob_label'].replace({
    'drob_pos': 'Positive',
    'drob_neu': 'Neutral',
    'drob_neg': 'Negative'
})

sentiment_df.head()


,ticker,pub_date,headline,vader_pos,vader_neu,vader_neg,vader_label,drob_neg,drob_neu,drob_pos,drob_label
0,AAPL,2015-04-07,M.B.A. Programs That Get You Where You Want to Go,0.126,0.874,0.0,Positive,0.021400,0.790080,0.188520,Neutral
1,AAPL,2015-04-14,What We’re Reading,0.000,1.000,0.0,Neutral,0.055277,0.798512,0.146211,Neutral
2,AAPL,2015-04-13,IBM Creates Watson Health to Analyze Medical Data,0.231,0.769,0.0,Positive,0.015715,0.696537,0.287748,Neutral
3,AAPL,2015-04-22,What’s That on Beyoncé’s Wrist? Let Me Guess ....,0.000,1.000,0.0,Neutral,0.042261,0.861066,0.096673,Neutral
4,AAPL,2015-04-01,Daily Report: Tech Leaders Come Together to Op...,0.000,1.000,0.0,Neutral,0.506060,0.291502,0.202438,Negative


In [ ]:
# import tensorflow as tf

# def drob_polarity_scores(sentence):
#     encoded_text = tokenizer(sentence, return_tensors='tf')
    
#     output = drob_model(encoded_text)
#     scores = output.logits[0].numpy()
#     scores = tf.nn.softmax(scores).numpy()
    
#     scores_dict = {
#         'drob_neg': scores[0],
#         'drob_neu': scores[1],
#         'drob_pos': scores[2]
#     }
#     return scores[0], scores[1], scores[2]

# sentiment_df[['drob_pos','drob_neg','drob_neu']] = sentiment_df['headline'].apply(drob_polarity_scores).apply(pd.Series)

# # Add label based on largest (neg/neu/pos)
# sentiment_df['drob_label'] = sentiment_df.apply(lambda x:'Negative' if x['drob_neg'] >
#                      x['drob_neu'] and x['drob_neu'] > x['drob_pos'] else ('Neutral' if x['drob_neu']>x['drob_pos'] else 'Positive'),axis=1)

# sentiment_df.head() 


In [52]:
# Save the DataFrame to a CSV file just in case 
sentiment_df.to_csv('apple_sentiment_analysis_results.csv', index=False)


### 3. Transformer (DeBerta)


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("tammiloveshf/Fin_DeBerta")
deb_model = AutoModelForSequenceClassification.from_pretrained("tammiloveshf/Fin_DeBerta")

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

c:\Users\Tammy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tammy\.cache\huggingface\hub\models--tammiloveshf--Fin_DeBerta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [11]:
import torch

# Function to get sentiment and probabilities for negative, neutral, and positive
def get_sentiment(headline):
    # Tokenize the input headline
    encoded_text = tokenizer(headline, return_tensors='pt', padding=True, truncation=True)
    
    # Get the model's output (logits)
    output = deb_model(**encoded_text)
    
    # Apply softmax to get probabilities for each class (negative, neutral, positive)
    probs = torch.nn.functional.softmax(output.logits, dim=-1)
    
    # Extract sentiment scores (negative, neutral, positive)
    deb_neg, deb_neu, deb_pos = probs[0].detach().numpy()
    
    # Return sentiment scores as well as the label
    return deb_neg, deb_neu, deb_pos, 'Negative' if deb_neg > deb_neu and deb_neg > deb_pos else ('Neutral' if deb_neu > deb_pos else 'Positive')

# Apply the function to the 'headline' column and unpack the values into separate columns
sentiment_df[['deb_pos', 'deb_neg', 'deb_neu', 'deb_label']] = sentiment_df['headline'].apply(get_sentiment).apply(pd.Series)

# Display the first few rows of the DataFrame to verify the results
sentiment_df.head(10)


KeyboardInterrupt: 

In [12]:
sentiment_df.head()

,ticker,pub_date,headline,vader_pos,vader_neu,vader_neg,vader_label,drob_neg,drob_neu,drob_pos,drob_label,deb_pos,deb_neg,deb_neu,deb_label
0,AAPL,2015-04-07,M.B.A. Programs That Get You Where You Want to Go,0.126,0.874,0.0,Positive,0.021400,0.790080,0.188520,Neutral,0.001556,0.910922,0.087523,Neutral
1,AAPL,2015-04-14,What We’re Reading,0.000,1.000,0.0,Neutral,0.055277,0.798512,0.146211,Neutral,0.012333,0.986333,0.001334,Neutral
2,AAPL,2015-04-13,IBM Creates Watson Health to Analyze Medical Data,0.231,0.769,0.0,Positive,0.015715,0.696537,0.287748,Neutral,0.001769,0.969001,0.029229,Neutral
3,AAPL,2015-04-22,What’s That on Beyoncé’s Wrist? Let Me Guess ....,0.000,1.000,0.0,Neutral,0.042261,0.861066,0.096673,Neutral,0.059633,0.933838,0.006529,Neutral
4,AAPL,2015-04-01,Daily Report: Tech Leaders Come Together to Op...,0.000,1.000,0.0,Neutral,0.506060,0.291502,0.202438,Negative,0.910898,0.066995,0.022107,Negative


In [13]:
# Save the DataFrame to a CSV file just in case 
sentiment_df.to_csv('finetuned_sentiment_model.csv', index=False)

## Run abstract through all 5 models


In [5]:
#narrowing down to headlines and abstract
# Select the desired columns
sentiment_df = df[['ticker', 'pub_date', 'abstract']]
sentiment_df = sentiment_df.copy()  # Make a copy to avoid modifications on a slice
sentiment_df.head()

,ticker,pub_date,abstract
0,AAPL,2015-01-05,You can adjust your settings so that only cert...
1,AAPL,2015-01-05,It’s time to take note of what lies at the top...
2,AAPL,2015-01-06,
3,AAPL,2015-01-07,"With apps able to act as cameras, media player..."
4,AAPL,2015-01-08,Google’s app store is the largest in the world...


### 1. Vader

In [6]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trinatan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
# Run model
sia = SentimentIntensityAnalyzer()

# Function to get full sentiment scores (including probabilities)
def get_sentiment(text):
    # Get the sentiment scores dictionary (positive, neutral, negative, and compound scores)
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores

# Apply the sentiment analysis to the 'headline' column
sentiment_df['sentiment_scores'] = sentiment_df['abstract'].apply(get_sentiment)

# Separate out the individual sentiment probabilities into new columns
sentiment_df['vader_pos'] = sentiment_df['sentiment_scores'].apply(lambda x: x['pos'])
sentiment_df['vader_neu'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neu'])
sentiment_df['vader_neg'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neg'])

# Function to classify sentiment into Positive, Negative, or Neutral based on the compound score
def classify_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the compound score to classify sentiment
sentiment_df['vader_label'] = sentiment_df['sentiment_scores'].apply(lambda x: classify_sentiment(x['compound']))

# Display the result
# Drop the 'sentiment_scores' column from the DataFrame
sentiment_df = sentiment_df.drop(columns=['sentiment_scores'])

# Display the result
sentiment_df.head(5)


,ticker,pub_date,abstract,vader_pos,vader_neu,vader_neg,vader_label
0,AAPL,2015-01-05,You can adjust your settings so that only cert...,0.106,0.894,0.000,Positive
1,AAPL,2015-01-05,It’s time to take note of what lies at the top...,0.092,0.765,0.143,Negative
2,AAPL,2015-01-06,,0.000,0.000,0.000,Neutral
3,AAPL,2015-01-07,"With apps able to act as cameras, media player...",0.000,0.912,0.088,Negative
4,AAPL,2015-01-08,Google’s app store is the largest in the world...,0.000,1.000,0.000,Neutral


### 2. Transformer (distilRoBERTa)

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("frostedtrees/Fin_distilroberta")
drob_model = AutoModelForSequenceClassification.from_pretrained("frostedtrees/Fin_distilroberta", num_labels=3)  # Assuming 3 sentiments: positive, neutral, negative


/opt/anaconda3/envs/MLApps/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def predict_sentiment(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = drob_model(**inputs)
    
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)  # Convert logits to probabilities
    
    return probs[0].cpu().numpy()  # Returns an array of probabilities

# Apply prediction to DataFrame
sentiment_probs = sentiment_df['abstract'].apply(predict_sentiment)

# Create separate columns for sentiment probabilities
sentiment_df[['drob_neg', 'drob_neu', 'drob_pos']] = pd.DataFrame(sentiment_probs.tolist(), index=sentiment_probs.index)

# Determine final sentiment label based on highest probability and simplify label mapping
sentiment_df['drob_label'] = sentiment_df[['drob_neg', 'drob_neu', 'drob_pos']].idxmax(axis=1)
sentiment_df['drob_label'] = sentiment_df['drob_label'].replace({
    'drob_pos': 'Positive',
    'drob_neu': 'Neutral',
    'drob_neg': 'Negative'
})

sentiment_df.head()


,ticker,pub_date,abstract,vader_pos,vader_neu,vader_neg,vader_label,drob_neg,drob_neu,drob_pos,drob_label
0,AAPL,2015-01-05,You can adjust your settings so that only cert...,0.106,0.894,0.000,Positive,0.075523,0.807759,0.116718,Neutral
1,AAPL,2015-01-05,It’s time to take note of what lies at the top...,0.092,0.765,0.143,Negative,0.066431,0.663510,0.270059,Neutral
2,AAPL,2015-01-06,,0.000,0.000,0.000,Neutral,0.026959,0.898535,0.074505,Neutral
3,AAPL,2015-01-07,"With apps able to act as cameras, media player...",0.000,0.912,0.088,Negative,0.787576,0.128845,0.083579,Negative
4,AAPL,2015-01-08,Google’s app store is the largest in the world...,0.000,1.000,0.000,Neutral,0.593591,0.319998,0.086411,Negative


### 3. Transformer (DeBerta)


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("tammiloveshf/Fin_DeBerta")
deb_model = AutoModelForSequenceClassification.from_pretrained("tammiloveshf/Fin_DeBerta")

In [11]:
import torch

# Function to get sentiment and probabilities for negative, neutral, and positive
def get_sentiment(abstract):
    # Tokenize the input abstract
    encoded_text = tokenizer(abstract, return_tensors='pt', padding=True, truncation=True)
    
    # Get the model's output (logits)
    output = deb_model(**encoded_text)
    
    # Apply softmax to get probabilities for each class (negative, neutral, positive)
    probs = torch.nn.functional.softmax(output.logits, dim=-1)
    
    # Extract sentiment scores (negative, neutral, positive)
    deb_neg, deb_neu, deb_pos = probs[0].detach().numpy()
    
    # Return sentiment scores as well as the label
    return deb_neg, deb_neu, deb_pos, 'Negative' if deb_neg > deb_neu and deb_neg > deb_pos else ('Neutral' if deb_neu > deb_pos else 'Positive')

# Apply the function to the 'abstract' column and unpack the values into separate columns
sentiment_df[['deb_pos', 'deb_neg', 'deb_neu', 'deb_label']] = sentiment_df['abstract'].apply(get_sentiment).apply(pd.Series)

# Display the first few rows of the DataFrame to verify the results
sentiment_df.head(10)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,ticker,pub_date,abstract,vader_pos,vader_neu,vader_neg,vader_label,drob_neg,drob_neu,drob_pos,drob_label,deb_pos,deb_neg,deb_neu,deb_label
0,AAPL,2015-01-05,You can adjust your settings so that only cert...,0.106,0.894,0.000,Positive,0.075523,0.807759,0.116718,Neutral,0.000852,0.988287,0.010861,Neutral
1,AAPL,2015-01-05,It’s time to take note of what lies at the top...,0.092,0.765,0.143,Negative,0.066431,0.663510,0.270059,Neutral,0.001523,0.985242,0.013235,Neutral
2,AAPL,2015-01-06,,0.000,0.000,0.000,Neutral,0.026959,0.898535,0.074505,Neutral,0.072680,0.207977,0.719343,Positive
3,AAPL,2015-01-07,"With apps able to act as cameras, media player...",0.000,0.912,0.088,Negative,0.787576,0.128845,0.083579,Negative,0.025295,0.898772,0.075933,Neutral
4,AAPL,2015-01-08,Google’s app store is the largest in the world...,0.000,1.000,0.000,Neutral,0.593591,0.319998,0.086411,Negative,0.110893,0.643165,0.245942,Neutral
5,AAPL,2015-01-09,Facebook eventually reverts to the “Top Storie...,0.000,1.000,0.000,Neutral,0.094106,0.804785,0.101109,Neutral,0.080449,0.860113,0.059438,Neutral
6,AAPL,2015-01-12,,0.000,0.000,0.000,Neutral,0.026959,0.898535,0.074505,Neutral,0.072680,0.207977,0.719343,Positive
7,AAPL,2015-01-13,,0.000,0.000,0.000,Neutral,0.026959,0.898535,0.074505,Neutral,0.072680,0.207977,0.719343,Positive
8,AAPL,2015-01-14,"Apple, Google, Intel and Adobe say a new deal ...",0.070,0.800,0.130,Negative,0.258490,0.338560,0.402950,Positive,0.004473,0.043939,0.951588,Positive
9,AAPL,2015-01-14,"Apple, Google, Intel and Adobe are said to be ...",0.000,0.901,0.099,Negative,0.049437,0.630534,0.320029,Neutral,0.043262,0.134464,0.822274,Positive


In [12]:
# Export to csv
'../../../data/clean/multimodal_inputs/llm_text_data_processed_abstract_AAPL.json'
sentiment_df.to_csv('../../../data/clean/sentiment_analysis_results/finetuned_sentiment_analysis_abstract_AAPL.csv')
